# Genetická optimalizácia (GO) problému obchodného cestujúceho (TSP)

Tento notebook sa zaoberá GO (Genetic optimization) na TSP (Travelling salesman problem). Špeciálna pozornosť je venovaná skúmaniu crossover metód a ich porovnaniu. Súčasťou práce je zavedenie 4 nových crossover metód a ich porovnanie s metódami implementovanými v rámci kurzu.

## Ciele práce 

1. Definovanie nových crossover metód.
2. Hľadanie optimálnych kombinácii crossover, mutation a correction metód pri GO na TSPGrid(3, 3).
3. Tuning hyper-parametrov N, M, T_{1}, T_{2}.
4. Zhodnotenie vplyvu náhodného generovania počiatočných riešení vs. žravej heuristiky.
5. Záverečné zhodnotenie dosiahnutých výsledkov.

## 1. Nové crossover metódy

<b>MatchCrossoverMax() </b>

Pokiaľ sú rodičia (teda riešenia problému) vyjadrené ako vektory: $[x_{0},...,x_{n} ] $ a $[y_{0},...,y_{n} ] $, potom dieťa vyjadrené ako: $[z_{0},...,z_{n} ] $ vznikne nasledovne. 

In [1]:
# Import path to source directory (bit of a hack in Jupyter)
import sys
import os
pwd = %pwd
sys.path.append(os.path.join(pwd, os.path.join('..', 'src')))

# Ensure modules are reloaded on any change (very useful when developing code on the fly)
%load_ext autoreload
%autoreload 2

# Work on local computer... add path to source directory
sys.path.append('/home/benjamin/Documents/18heur-2020-master/src')

In [2]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

import matplotlib
%matplotlib notebook
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [3]:
from objfun_tsp import TSPGrid
tsp = TSPGrid(3, 3)

In [4]:
from heur_go import Crossover, UniformMultipoint, RandomCombination # crossover
from heur_go import MatchCrossoverMax, MatchCrossoverRnd, MisMatchCrossoverMax, MisMatchCrossoverRnd # new crossovers
from heur_go import GeneticOptimization # heuristics 
from heur_aux import Correction, MirrorCorrection, ExtensionCorrection # corrections 
from heur_aux import CauchyMutation # mutation

In [5]:
# tu podoplnat nove crossovers, corrections, ...

multipoints = [1, 2, 3]
crossovers = [
    {'crossover': MatchCrossoverMax(), 'name': 'MCM'},
    {'crossover': MatchCrossoverRnd(), 'name': 'MCR'},
    {'crossover': MisMatchCrossoverMax(), 'name': 'MMCM'},
    {'crossover': MisMatchCrossoverRnd(), 'name': 'MMCR'},
    {'crossover': Crossover(), 'name': 'mix'},
    {'crossover': RandomCombination(), 'name': 'rnd'}]
for multipoint in multipoints:
    crossover = {'crossover': UniformMultipoint(multipoint), 'name': 'uni{}'.format(multipoint)}
    crossovers.append(crossover)

corrections = [
    {'correction': Correction(tsp), 'name': 'vanilla'},
    {'correction': MirrorCorrection(tsp), 'name': 'mirror'},
    {'correction': ExtensionCorrection(tsp), 'name': 'extension'}]

parameters = [2, 3, 1]
mutations = []
for parameter in parameters:
    for correction in corrections:
        mutation = {'mutation': CauchyMutation(r=parameter, correction = correction['correction']), 'name': 'cauchy{}_'
                   .format(parameter)+correction['name']}
        mutations.append(mutation)

In [6]:
NUM_RUNS = 1000
maxeval = 100

In [7]:
results = pd.DataFrame()
for crossover in crossovers:
    for mutation in mutations:
        heur_name = 'GO_mut:({})_cro:{}'.format(mutation['name'], crossover['name'])
        runs = []
        for i in tqdm(range(NUM_RUNS), 'Testing {}'.format(heur_name)):
            run = GeneticOptimization(tsp, maxeval, N=5, M=15, Tsel1=1.0, Tsel2=0.5, 
                                      mutation=mutation['mutation'],
                                      crossover=crossover['crossover']).search()
            run['run'] = i
            run['heur'] = heur_name
            run['mutation'] = mutation['name']
            run['crossover'] = crossover['name']
            runs.append(run)

        res_df = pd.DataFrame(runs, columns=['heur', 'run', 'mutation', 'crossover','best_x', 'best_y', 'neval'])
        results = pd.concat([results, res_df], axis=0)

In [8]:
def rel(x):
    return len([n for n in x if n < np.inf])/len(x)
def mne(x):
    return np.mean([n for n in x if n < np.inf])
def feo(x):
    return mne(x)/rel(x)

In [11]:
results_pivot = results.pivot_table(
    index=['heur', 'mutation', 'crossover'],
    values=['neval'],
    aggfunc=(rel, mne, feo)
)['neval']
results_pivot = results_pivot.reset_index()
pd.set_option('display.max_rows', 500)
results_pivot.sort_values(by='feo')


/home/benjamin/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/benjamin/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,heur,mutation,crossover,feo,mne,rel
24,GO_mut:(cauchy1_vanilla)_cro:uni1,cauchy1_vanilla,uni1,255.620485,56.747748,0.222
18,GO_mut:(cauchy1_vanilla)_cro:MCM,cauchy1_vanilla,MCM,263.863636,58.050000,0.220
26,GO_mut:(cauchy1_vanilla)_cro:uni3,cauchy1_vanilla,uni3,273.893219,57.791469,0.211
25,GO_mut:(cauchy1_vanilla)_cro:uni2,cauchy1_vanilla,uni2,285.782272,58.585366,0.205
23,GO_mut:(cauchy1_vanilla)_cro:rnd,cauchy1_vanilla,rnd,292.567016,58.805970,0.201
76,GO_mut:(cauchy3_vanilla)_cro:mix,cauchy3_vanilla,mix,294.246219,54.141304,0.184
22,GO_mut:(cauchy1_vanilla)_cro:mix,cauchy1_vanilla,mix,298.384395,58.781726,0.197
51,GO_mut:(cauchy2_vanilla)_cro:uni1,cauchy2_vanilla,uni1,314.847241,57.302198,0.182
45,GO_mut:(cauchy2_vanilla)_cro:MCM,cauchy2_vanilla,MCM,323.553097,54.680473,0.169
78,GO_mut:(cauchy3_vanilla)_cro:uni1,cauchy3_vanilla,uni1,328.567281,54.542169,0.166


vsetky vanilla correction su lepsie nez extension a mirror, takze pri dalsom skumani uvazujeme iba tie

MCM lepsie ako MCR, MMCM lepsie ako MMCR

pre uni*, MCM, rnd je to cauchy1
pre MMCM, mix je to cauchy3

In [12]:
N = [3, 5, 10, 15]
T1 = [0.5, 1, 5]
T2 = [0.1, 0.4]

In [13]:
crossovers_mutations = [
    {'crossover': UniformMultipoint(1), 'mutation': CauchyMutation(r=1, correction = Correction(tsp)), 'name':
    'uni1_cauchy1'},
    {'crossover': MatchCrossoverMax(), 'mutation': CauchyMutation(r=1, correction = Correction(tsp)), 'name':
    'MCM_cauchy1'},
    {'crossover': MisMatchCrossoverMax(), 'mutation': CauchyMutation(r=3, correction = Correction(tsp)), 'name':
    'MMCM_cauchy3'},
    {'crossover': RandomCombination(), 'mutation': CauchyMutation(r=1, correction = Correction(tsp)), 'name':
    'rnd_cauchy1'}
]

In [14]:
results_2 = pd.DataFrame()
for n in N:
    for temp1 in T1:
        for temp2 in T2:
            for cr_mut in crossovers_mutations:
                heur_name = 'GO_{}_N:{}_T1:{}_T2:{}'.format(cr_mut['name'], n, temp1, temp2)
                runs = []
                for i in tqdm(range(NUM_RUNS), 'Testing {}'.format(heur_name)):
                    run = GeneticOptimization(tsp, maxeval, N=n, M=3*n, Tsel1=temp1, Tsel2=temp2, 
                                              mutation=cr_mut['mutation'],
                                              crossover=cr_mut['crossover']).search()
                    run['run'] = i
                    run['heur'] = heur_name
                    run['n'] = n
                    run['temp1'] = temp1
                    run['temp2'] = temp2
                    runs.append(run)

                res_df = pd.DataFrame(runs, columns=['heur', 'run', 'n','temp1', 'temp2', 
                                                     'best_x', 'best_y', 'neval'])
                results_2 = pd.concat([results_2, res_df], axis=0)

In [16]:
results_pivot = results_2.pivot_table(
    index=['heur', 'n', 'temp1', 'temp2'],
    values=['neval'],
    aggfunc=(rel, mne, feo)
)['neval']
results_pivot = results_pivot.reset_index()
results_pivot.sort_values(by='feo')

/home/benjamin/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/benjamin/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,heur,n,temp1,temp2,feo,mne,rel
18,GO_MCM_cauchy1_N:5_T1:0.5_T2:0.1,5,0.5,0.1,228.224000,57.056000,0.250
21,GO_MCM_cauchy1_N:5_T1:1_T2:0.4,5,1.0,0.4,243.343079,58.158996,0.239
61,GO_rnd_cauchy1_N:3_T1:0.5_T2:0.4,3,0.5,0.4,248.966942,54.772727,0.220
85,GO_uni1_cauchy1_N:3_T1:0.5_T2:0.4,3,0.5,0.4,254.167556,57.696035,0.227
14,GO_MCM_cauchy1_N:3_T1:1_T2:0.1,3,1.0,0.1,258.766465,54.858491,0.212
92,GO_uni1_cauchy1_N:5_T1:1_T2:0.1,5,1.0,0.1,263.234181,56.595349,0.215
20,GO_MCM_cauchy1_N:5_T1:1_T2:0.1,5,1.0,0.1,266.350617,59.928889,0.225
84,GO_uni1_cauchy1_N:3_T1:0.5_T2:0.1,3,0.5,0.1,269.499343,53.091371,0.197
15,GO_MCM_cauchy1_N:3_T1:1_T2:0.4,3,1.0,0.4,270.577037,55.468293,0.205
13,GO_MCM_cauchy1_N:3_T1:0.5_T2:0.4,3,0.5,0.4,271.286561,57.784038,0.213
